In [ ]:
import cv2

# Initialize the camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not capture frame.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Load the face cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Get the region of interest (ROI) which is the detected face
        roi_gray = gray[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Apply Sobel filters to detect horizontal and vertical edges
        sobelx = cv2.Sobel(roi_gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(roi_gray, cv2.CV_64F, 0, 1, ksize=5)

        # Convert back to uint8
        sobelx = cv2.convertScaleAbs(sobelx)
        sobely = cv2.convertScaleAbs(sobely)

        # Combine the horizontal and vertical edges
        edges_horizontal = cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0)
        edges_vertical = sobely  # Extract vertical edges only

        # Display the edges
        cv2.imshow('Horizontal Edges', edges_horizontal)
        cv2.imshow('Vertical Edges', edges_vertical)

    # Display the frame
    cv2.imshow('Camera', frame)

    # Wait for the 's' key to be pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        # Save the snapshot
        cv2.imwrite('snapshot.png', frame)
        print("Snapshot saved.")
        
        # Load and display the snapshot
        snapshot = cv2.imread('snapshot.png')
        cv2.imshow('Snapshot', snapshot)
        cv2.waitKey(0)  # Wait indefinitely until a key is pressed to close the snapshot window
        
        # Check if the window exists before attempting to destroy it
        if cv2.getWindowProperty('Snapshot', cv2.WND_PROP_VISIBLE) > 0:
            cv2.destroyWindow('Snapshot')
            
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()